In [18]:
# notebook settings
%load_ext autoreload
%autoreload 2
# %matplotlib notebook

# external imports
import numpy as np
import sympy as sp
# from scipy.linalg import solve_discrete_are
import matplotlib.pyplot as plt

# internal imports
from controller import controller
from nonlinear_dynamics import simulate
from mld_dynamics import mld, h
from visualizer import vis, animate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Open-loop solution

In [19]:
# initial push towards the right wall
x0 = np.array([0., 0., 1., 0.])

In [20]:
# solve in open loop
# options = {'OptimalityTol': 1e-06, 'Presolve': 0, 'Heuristics': 0}
solution, leaves, nodes, solver_time = controller.feedforward(
    x0,
    printing_period=None,
    gurobi_params={'OutputFlag': 0, 'Method': 1} # dual simplex
#     draw_label='Cart pole with walls'
)
solver_time
solution.objective

0.06925696807132983

import cProfile
cProfile.run('controller.feedforward(x0, printing_period=None)')

In [ ]:
options = {'OutputFlag': 1, 'MIPGap': 0, 'Presolve': 0, 'Heuristics': 0}
x_gurobi, obj_gurobi, nodes_gurobi, solver_time = controller.feedforward_gurobi(x0, options)
solver_time

In [ ]:
vis.jupyter_cell()

In [ ]:
animate(solution.variables['x'], h)

## Solve with and without warm start

In [176]:
e0 = np.random.randn(mld.nx) * 0.
uc0 = solution.variables['uc'][0]
ub0 = solution.variables['ub'][0]
x1 = solution.variables['x'][1]
warm_start_, ws_time_ = controller.construct_warm_start(leaves, x0, uc0, ub0, e0)
ws_time_

0.008633852005004883

In [168]:
import cProfile
e0 = np.random.randn(mld.nx) * 0.
uc0 = solution.variables['uc'][0]
ub0 = solution.variables['ub'][0]
cProfile.run('controller.construct_warm_start(leaves, x0, uc0, ub0, e0)')

         5734 function calls in 0.015 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(concatenate)
       77    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(copyto)
        1    0.000    0.000    0.015    0.015 <string>:1(<module>)
       77    0.000    0.000    0.000    0.000 branch_and_bound.py:12(__init__)
       77    0.003    0.000    0.004    0.000 controller.py:300(_get_bound_binaries)
        1    0.001    0.001    0.014    0.014 controller.py:454(construct_warm_start)
       77    0.001    0.000    0.001    0.000 controller.py:516(<dictcomp>)
       81    0.000    0.000    0.001    0.000 controller.py:526(_retain_leaf)
      399    0.000    0.000    0.000    0.000 controller.py:544(<genexpr>)
       77    0.001    0.000    0.002    0.000 controller.py:546(_shift_dual_variables)
       77    0.002    0.000    0.009   

In [ ]:
u0 = np.concatenate((uc0, ub0))
x1 = mld.A.dot(x0) + mld.B.dot(u0) + e0
solution, leaves, nodes, solver_time = controller.feedforward(
    x1,
    warm_start=warm_start,
    gurobi_options={'Method': 1}, # dual simplex
#     draw_label='Cart pole with walls warm start'
)
solver_time

In [ ]:
solution, leaves, nodes, solver_time = controller.feedforward(x1)
solver_time